##1. Lista de librerias

In [2]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##2 Funciones

In [5]:
%run '/content/drive/MyDrive/Colab Notebooks/Tesis_Functions.ipynb'

Selectkbest_features Loaded
rfe_features Loaded
Integración de Variables Loaded
evaluacion de los modelos Loaded
Pca_components Loaded
Improtancia con XGBoost y Random forest Loaded
XG_BOOST Regression Loaded
Random Forest Loaded
quitar_tildes Loaded
completar_datos_serie_tiempo Loaded
eliminar_filas_nan Loaded
cambiar_numeros_menores Loaded
normalize_variables Loaded
calcular_crecimiento_ingresos Loaded
obtener_variables_correlacionadas Loaded
generar_componentes_principales Loaded
rfe_with_elasticnet Loaded
calcular_crecimiento_promedio_por_categoria_Predial Loaded
categorizar_por_boxplot Loaded
calcular_crecimiento_promedio_por_categoria Loaded
limitar_atipicos Loaded
crear_dummies Loaded


##3. Cargar Datos Clasificación

In [6]:
#Lista de municipios (Regresión: ok; Clasifficación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1a9IfZDuiqbhh2iefH8BMarHL8IXWpayF'
df = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')
Municipios = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')

In [7]:
#Impuesto predial por municipio (Regresión: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1c58chHv4Gt4vEq0trAkjdJdFza_Xlce9'
Predial_Transpuesto = pd.read_csv(archivo_csv, sep=';')

#Impuesto predial por municipio (Clasificación: ok)
Cat_Clasificiacion=calcular_crecimiento_promedio_por_categoria_Predial(Predial_Transpuesto,2008,2010,2018,2020)
Cat_Clasificiacion=Cat_Clasificiacion.to_frame().reset_index()
Categorias_Predial=categorizar_por_boxplot(Cat_Clasificiacion,Cat_Clasificiacion['Predial'])

In [8]:
#IPS por municipio (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1FnlTto45_Uf2kfgplRG3BIGO5cuwowpe'
IPS = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')
IPS = IPS.fillna(0)

In [9]:
#Información agricola por municipio (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1VPjkJTYKuUYzK6Gy5Lgf_oUKbLEjJ0ci'
Agricultura = pd.read_csv(archivo_csv, sep=';')
Agricultura = Agricultura.fillna(1)
lista_columnas = Agricultura[Agricultura.columns.difference(['Municipio','Year'])].columns.tolist()
Agricultura=calcular_crecimiento_promedio_por_categoria(Agricultura,lista_columnas,2008,2010,2018,2020)
Agricultura=limitar_atipicos(Agricultura, lista_columnas)
Agricultura=Agricultura.reset_index()

In [10]:
# Dato de internet por municipio (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1_RlmOEByjtf5LZZLvijGvExy1UzpgqJt'
Internet = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')
Internet=Internet[['COD_MUNICIPIO','Indice_internet_2020']]
Internet = Internet.fillna(0)

In [11]:
#Seguridad por Municipio (Clasificación: ok)

archivo_csv = 'https://drive.google.com/uc?export=download&id=15rEU68edXPCII2V8m73tGph-B7b73nJs'
Seguridad_cat = pd.read_csv(archivo_csv, sep=',')
Seguridad_cat.fillna(1)
Seguridad_cat = Seguridad_cat.replace(to_replace=0, value=1)
columnas_like=['Seguridad_Abigeato_', 'Seguridad_Amenazas_', 'Seguridad_Delitos_Sexuales_', 'Seguridad_Extorsion_', 'Seguridad_homicidios_', 'Seguridad_T_Hurto_', 'Seguridad_hurto_a_comercio_',
 'Seguridad_T_homicidios_en_accidente_de_transito_', 'Seguridad_hurto_a_entidades_financieras_', 'Seguridad_hurto_a_residencias_', 'Seguridad_Hurto_Motocicleta_',
 'Seguridad_Lensiones_accidente_Transito_', 'Seguridad_lesiones_personales_', 'Seguridad_pirateria_terrestre_']


Seguridad_cat.rename(columns={'Id_Municipio_Dane': 'Codigo'}, inplace=True) #Cambio de nombre de columna
columna_puntual = Seguridad_cat['Codigo']
Seguridad_cat_2=pd.DataFrame(Seguridad_cat['Codigo'])

for columna in columnas_like:
    Fiscal_Temp=Seguridad_cat.filter(like=columna).copy()
    Fiscal_Temp['Codigo']=columna_puntual
    temp=pd.DataFrame(year_columnas(Fiscal_Temp, columna,'Codigo',2010,2012,2018,2020)['Industria'])
    Seguridad_cat_2[columna]=temp['Industria']
Seguridad_cat=Seguridad_cat_2
Seguridad_cat_2=[]

archivo_csv = 'https://drive.google.com/uc?export=download&id=148yrW8j8ePSXUqr8g39N_ghKYzx6DHhh'
Seguridad_total = pd.read_csv(archivo_csv, sep=',')

Seguridad_total.fillna(1)
Seguridad_total = Seguridad_total.replace(to_replace=0, value=1)
columnas_like=['Seguridad_']
Seguridad_total.rename(columns={'Id_Municipio_Dane': 'Codigo'}, inplace=True) #Cambio de nombre de columna
columna_puntual = Seguridad_total['Codigo']
Seguridad_total_2=pd.DataFrame(Seguridad_total['Codigo'])

for columna in columnas_like:
    Fiscal_Temp=Seguridad_total.filter(like=columna).copy()
    Fiscal_Temp['Codigo']=columna_puntual
    temp=pd.DataFrame(year_columnas(Fiscal_Temp, columna,'Codigo',2010,2012,2018,2020)['Industria'])
    Seguridad_total_2[columna]=temp['Industria']
Seguridad_total=Seguridad_total_2
Seguridad_total_2=[]

#archivo_csv = 'https://drive.google.com/uc?export=download&id=1h_UyaqBznDchxi6BREuR5SJYIZSdcsyZ'
#Seguridad_genero = pd.read_csv(archivo_csv, sep=',')

In [12]:
#Turismo por municipio (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1Oi4u9yHS7lwDZhD2UhlR0eJHs1cChKgx'
Turismo= pd.read_csv(archivo_csv, sep=';', encoding='latin-1')
Turismo = Turismo.fillna(0)

In [13]:
#Ingresos por Banca (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1DhjRoW8k3sYuD32Sj_yKsfUDPSUKDPYl'
Bancos= pd.read_csv(archivo_csv, sep=';',)
Bancos = eliminar_filas_nan(Bancos, 'Etiquetas de fila')
Bancos = cambiar_numeros_menores(Bancos, 100, 'Etiquetas de fila')
Bancos=year_columnas(Bancos, 'Bancos_','Etiquetas de fila',2008,2010,2018,2020)

In [ ]:
#Ingresos Industria y Comercio (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1fRK0My402q1VQkbZq62UOh1ssZTIIoH6'
Industria= pd.read_csv(archivo_csv, sep=';')
Industria = eliminar_filas_nan(Industria, 'Codigo')
Industria = cambiar_numeros_menores(Industria, 100, 'Codigo')
Industria=year_columnas(Industria, 'Industria_Comercio_','Codigo',2008,2010,2018,2020)

In [22]:
#Desempeño_Fiscal (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=13ggqFSiTvguqmKx7IX6J3Ia6Q9ktM_Cn'
Fiscal= pd.read_csv(archivo_csv, sep=';')


columnas_like=['Fiscal_Autofinanciacion_','Fiscal_Dependencia_Nación_','Fiscal_Generación_Recursos_','Fiscal_Magnitud_Iinversión_','Fiscal_Capacidad_ahorro_','Fiscal_Indicador_desempeño_Fiscal_']
columna_puntual = Fiscal['Código']
Fiscal_2=pd.DataFrame(Fiscal['Código'])

for columna in columnas_like:
    Fiscal_Temp=Fiscal.filter(like=columna).copy()
    Fiscal_Temp['Código']=columna_puntual
    temp=pd.DataFrame(year_columnas(Fiscal_Temp, columna,'Código',2008,2010,2017,2018)['Industria'])
    Fiscal_2[columna]=temp['Industria']


In [25]:
#Cerca_Ciudades (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1TDJYfmPvz1j9hR4fPUjKxQk-8j-yBtoV'
Ciudad= pd.read_csv(archivo_csv, sep=';')
columns_to_dummy=['Ciudad_Ley_617','Ciudad_Cerca_Ciudad','Ciudades_Capitales']
Ciudad=crear_dummies(Ciudad, columns_to_dummy)


In [29]:
#Probreza (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1zKt6krij1NhTHzi7LyXweJtbqSSolxM2'
Pobreza_Total= pd.read_csv(archivo_csv, sep=';')
columnas_like=['Pobreza_Total_','Pobreza_Cabeceras_','Pobreza_Rural_']
columna_puntual = Pobreza_Total['Codigo']
Pobreza_Total_2=pd.DataFrame(Pobreza_Total['Codigo'])

for columna in columnas_like:
    Fiscal_Temp=Pobreza_Total.filter(like=columna).copy()
    Fiscal_Temp['Codigo']=columna_puntual
    temp=pd.DataFrame(year_columnas(Fiscal_Temp, columna,'Codigo',2008,2008,2018,2018)['Industria'])
    Pobreza_Total_2[columna]=temp['Industria']
Pobreza_Total=Pobreza_Total_2
Pobreza_Total_2=[]

archivo_csv = 'https://drive.google.com/uc?export=download&id=1FLBP6ZKATfgYRl_pBk7FzY4UjDe4WWqM'
Pobreza_Detalle= pd.read_csv(archivo_csv, sep=';')

columnas_like=['Probreza_Analfabetismo_', 'Probreza_Bajo logro educativo_', 'Probreza_Barreras a servicios para cuidado de la primera infancia_', 'Probreza_Barreras de acceso a servicios de salud_',
 'Probreza_Tasa de dependecia económica_', 'Probreza_Hacinamiento crítico_', 'Probreza_Inadecuada eliminación de excretas_', 'Probreza_Inasistencia escolar_', 'Probreza_Material inadecuado de paredes exteriores_', 'Probreza_Material inadecuado de pisos_', 'Probreza_Rezago escolar_', 'Probreza_Sin acceso a fuente de agua mejorada_', 'Probreza_Sin aseguramiento en salud_', 'Probreza_Trabajo infantil_', 'Probreza_Trabajo informal_']

columna_puntual = Pobreza_Detalle['Codigo']
Pobreza_Detalle_2=pd.DataFrame(Pobreza_Detalle['Codigo'])

for columna in columnas_like:
    Fiscal_Temp=Pobreza_Detalle.filter(like=columna).copy()
    Fiscal_Temp['Codigo']=columna_puntual
    temp=pd.DataFrame(year_columnas(Fiscal_Temp, columna,'Codigo',2008,2008,2018,2018)['Industria'])
    Pobreza_Detalle_2[columna]=temp['Industria']
Pobreza_Detalle=Pobreza_Detalle_2
Pobreza_Detalle_2=[]

In [33]:
#Educacion (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1kyeKQcaFKBzf9OSxaD9LQ2AMbOZz17f9'
Educacion= pd.read_csv(archivo_csv, sep=';')


In [35]:
#Automotores (Clasificación: ok) ingreso por pago de autos
archivo_csv = 'https://drive.google.com/uc?export=download&id=1tdw20Uh2QQKETgWzGBwu9vygsxn3zaMT'
Autos= pd.read_csv(archivo_csv, sep=';',)
Autos = eliminar_filas_nan(Autos, 'Codigo')
Autos = cambiar_numeros_menores(Autos, 100, 'Codigo')
Autos=year_columnas(Autos, 'Autos_','Codigo',2008,2010,2018,2020)


In [37]:
#Descripcion_Predios (Clasificación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=17SX1r3FRC0C7Zsl8AyEm7RQ0zZ2UTIJa'
Lotes= pd.read_csv(archivo_csv, sep=',',)
Lotes = Lotes.fillna(0)

In [ ]:
#Terridata  (Clasificación: ok)

##Proceso se corrió y por tiempo de ejecución se cargó procesamiento de el dive
#archivo_csv = 'https://drive.google.com/uc?export=download&id=1_OA8ZG4ct75lERCJ19pEZ6BMWkR5yhiT'
#Terridata_Class= pd.read_csv(archivo_csv, sep=',',)
#Terridata_Class=Terridata_Class.replace(0, np.nan)
#columnas_a_eliminar = Terridata_Class.columns[Terridata_Class.isna().all()]
#Terridata_Class= Terridata_Class.drop(columnas_a_eliminar, axis=1)

#completa datos por interpolación, se demora 14 minutos en correr
#Terridata_Class=completar_datos_serie_tiempo(Terridata_Class,2008,2020)


#ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Terridata_Class_interpolada.csv'
#Terridata_Class.to_csv(ruta_archivo, index=False)


archivo_csv = 'https://drive.google.com/uc?export=download&id=1z-XajNX0Ia_BvcTT4hL3R39ySUCW7rTQ'
Terridata_Class= pd.read_csv(archivo_csv, sep=',',)

lista_columnas = [col for col in Terridata_Class.columns.tolist() if col not in ['Year', 'Category']]
Terridata_Class.rename(columns={'Category': 'Municipio'}, inplace=True)
Terridata_Class=calcular_crecimiento_promedio_por_categoria(Terridata_Class, lista_columnas, 2008, 2010, 2018, 2020)

columnas_a_eliminar = Terridata_Class.columns[Terridata_Class.isna().all()]
Terridata_Class= Terridata_Class.drop(columnas_a_eliminar, axis=1)
Terridata_Class= Terridata_Class.fillna(0)

Terridata_Class = Terridata_Class.drop('Municipio', axis=1)
Terridata_Class = Terridata_Class.reset_index()


In [ ]:
Cat_Clasificiacion.rename(columns={'CODIGO_DANE': 'Id_Municipio_Dane'}, inplace=True)
IPS.rename(columns={'ID_Municipio': 'Id_Municipio_Dane'}, inplace=True)
Agricultura.rename(columns={'index': 'Id_Municipio_Dane'}, inplace=True)
Internet.rename(columns={'COD_MUNICIPIO': 'Id_Municipio_Dane'}, inplace=True)
Seguridad_cat.rename(columns={'Codigo': 'Id_Municipio_Dane'}, inplace=True)
Seguridad_total.rename(columns={'Codigo': 'Id_Municipio_Dane'}, inplace=True)
Turismo.rename(columns={'id_dane_municipio': 'Id_Municipio_Dane'}, inplace=True)
Bancos.rename(columns={'Municipio': 'Id_Municipio_Dane'}, inplace=True)
Industria.rename(columns={'Municipio': 'Id_Municipio_Dane'}, inplace=True)
Fiscal_2.rename(columns={'Código': 'Id_Municipio_Dane'}, inplace=True)
Ciudad.rename(columns={'Código': 'Id_Municipio_Dane'}, inplace=True)
Pobreza_Total.rename(columns={'Codigo': 'Id_Municipio_Dane'}, inplace=True)
Pobreza_Detalle.rename(columns={'Codigo': 'Id_Municipio_Dane'}, inplace=True)
Educacion.rename(columns={'Codigo': 'Id_Municipio_Dane'}, inplace=True)
Autos.rename(columns={'Municipio': 'Id_Municipio_Dane'}, inplace=True)
Lotes.rename(columns={'Id_Municipio_Dane': 'Id_Municipio_Dane'}, inplace=True)
Terridata_Class.rename(columns={'index': 'Id_Municipio_Dane'}, inplace=True)

#4. Cargar Datos Regresión

para trabajar en regresión
1. info agricola
2. seguridad
3. Bancos
4. industria
5. desempeño_fiscal
6. cerca a ciudaes
7. Educación
8.  Autos

In [ ]:
#Terridata
archivo_csv = 'https://drive.google.com/uc?export=download&id=1_OA8ZG4ct75lERCJ19pEZ6BMWkR5yhiT'
Terridata= pd.read_csv(archivo_csv, sep=',',)


In [ ]:
#Ceracion de columna objetivo, crecimeinto 3 años en el futoro
columnas_a_eliminar = ['NOMBRE_ENTIDAD']
Predial_df=Predial_Transpuesto.drop(columnas_a_eliminar, axis=1)
Predial_Y = calcular_crecimiento_ingresos(Predial_df,3,'CODIGO_DANE','Predial','Año')

In [ ]:
# Creacion columna mes anterior de predial
columnas_a_eliminar = ['NOMBRE_ENTIDAD']
Predial_df=Predial_Transpuesto.drop(columnas_a_eliminar, axis=1)
Predial_mes_anterior = calcular_crecimiento_ingresos(Predial_df,-1,'CODIGO_DANE','Predial','Año')
Predial_mes_anterior = Predial_mes_anterior.rename(columns={'Crecimiento': 'Crecimiento_mes_anterior'})

# Creacion columna 2 meses atras de predial
columnas_a_eliminar = ['NOMBRE_ENTIDAD']
Predial_df=Predial_Transpuesto.drop(columnas_a_eliminar, axis=1)
Predial_mes_anterior_2 = calcular_crecimiento_ingresos(Predial_df,-2,'CODIGO_DANE','Predial','Año')
Predial_mes_anterior_2 = Predial_mes_anterior_2.rename(columns={'Crecimiento': 'Crecimiento_mes_anterior_2'})

# Creacion columna 3 meses atras de predial
columnas_a_eliminar = ['NOMBRE_ENTIDAD']
Predial_df=Predial_Transpuesto.drop(columnas_a_eliminar, axis=1)
Predial_mes_anterior_3 = calcular_crecimiento_ingresos(Predial_df,-3,'CODIGO_DANE','Predial','Año')
Predial_mes_anterior_3 = Predial_mes_anterior_3.rename(columns={'Crecimiento': 'Crecimiento_mes_anterior_3'})


In [ ]:
nombres_columnas = Terridata.columns.tolist()
nombres_columnas = pd.DataFrame(nombres_columnas)


In [ ]:
Desempeno_Seguridad

In [ ]:
Economia_ind_manufacturera
Economia_construccion
Finanzas_%gasto_total_inversion
Desempeno_Salud
Censo_pob_urbana
Economia_Primer_Cultivo_permanente(dumi)
Finanzas_agua
OT_Avaluo_catastral_total
OT_Total_predios
OT_Valorizacion

##4. Creación Base 0 Clasificación

### 4.1. Unir data frames de categorización

In [ ]:
#Quitar tildes de nomblre de municio
df['Nombre_Municipio'] = df['Nombre_Municipio'].apply(quitar_tildes)

In [ ]:
#Unir data sets Id_Municipio_Dane CODIGO_DANE

df_unido=df.merge(Cat_Clasificiacion, on='Id_Municipio_Dane', how='inner')
df_unido=df_unido.merge(IPS, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Agricultura, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Internet, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Seguridad_cat, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Seguridad_total, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Turismo, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Bancos, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Industria, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Fiscal_2, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Ciudad, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Pobreza_Total, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Pobreza_Detalle, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Educacion, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Autos, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Lotes, on='Id_Municipio_Dane', how='left')
df_unido=df_unido.merge(Terridata_Class, on='Id_Municipio_Dane', how='left')
#df_unido = df_unido.fillna(0)

In [ ]:
#Borrar columans con solo Nan (1979 columnas)
columnas_a_eliminar = df_unido.columns[df_unido.isna().all()]
df_unido= df_unido.drop(columnas_a_eliminar, axis=1)
df_unido = df_unido.fillna(0)

In [ ]:
non_numeric_variables = df_unido .select_dtypes(exclude=[np.number]).columns.tolist()
non_numeric_variables

['Nombre_Municipio', 'Nombre_Departamento', 'recomendacion']

###4.2. Normzalizar Varibles

In [ ]:
# Normalizar variables
lista_cat=['Id_Municipio_Dane','Nombre_Municipio','Id_Departamento_Dane','Nombre_Departamento','Predial','recomendacion']
Categorias=df_unido [lista_cat]
Normalizar=df_unido.drop(lista_cat, axis=1)
df_unido_Normalizada=normalize_variables(Normalizar)
df_unido_Normalizada=Categorias.join(df_unido_Normalizada)

In [ ]:
df_unido_Normalizada

,Id_Municipio_Dane,Nombre_Municipio,Id_Departamento_Dane,Nombre_Departamento,Predial,recomendacion,IPS_Nivel_1,IPS_Nivel_2,IPS_Nivel_3,IPS_Clinica,...,vivienda_servicios_publicos_CoberturaEnergiaElectricaUrbanaCenso,vivienda_servicios_publicos_CoberturaGasNaturalCenso,vivienda_servicios_publicos_CoberturaGasNaturalRuralCenso,vivienda_servicios_publicos_CoberturaGasNaturalUrbanoCenso,vivienda_servicios_publicos_CoberturaInternetCenso,vivienda_servicios_publicos_CoberturaInternetRuralCenso,vivienda_servicios_publicos_CoberturaInternetUrbanaCenso,vivienda_servicios_publicos_DeficitCualitativoViviendaCenso,vivienda_servicios_publicos_DeficitCuantitativoViviendaCenso,vivienda_servicios_publicos_PenetracionBandaAncha
0,5001,Medellin,5,Antioquia,133.859306,Media,0.153846,0.50,0.222222,0.3625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003260
1,5002,Abejorral,5,Antioquia,104.386980,Media,0.076923,0.00,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026511
2,5004,Abriaqui,5,Antioquia,92.937014,Media,0.000000,0.00,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004336
3,5021,Alejandria,5,Antioquia,212.680226,Media,0.076923,0.00,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004397
4,5030,Amaga,5,Antioquia,192.535616,Media,0.076923,0.00,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,97666,Taraira,97,Vaupés,1406.942589,Muy alta,0.000000,0.00,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000118
1098,99001,Puerto Carreno,99,Vichada,253.440132,Alta,0.000000,0.25,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037458
1099,99524,La Primavera,99,Vichada,1493.291755,Muy alta,0.000000,0.00,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001127
1100,99624,Santa Rosalia,99,Vichada,763.878456,Muy alta,0.000000,0.00,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


###4.3. Reducir variable scon alta correlación

In [ ]:
###Analisis de las variables con alta correlación para convertir en componentes principales

#Genera lista con las variables correlacionadas por grupos
lista_cat=['Id_Municipio_Dane','Nombre_Municipio','Id_Departamento_Dane','Nombre_Departamento','Predial','recomendacion']
df_unido_corr=df_unido_Normalizada.drop(lista_cat, axis=1)
grupos_variables=obtener_variables_correlacionadas(df_unido_corr,0.8)
# Contar las categorías
conteo_categorias = grupos_variables['Grupo'].value_counts()

# Filtrar las categorías con más de un registro
categorias_mas_de_un_registro = conteo_categorias[conteo_categorias > 1]
categorias_mas_de_un_registro=pd.DataFrame(categorias_mas_de_un_registro)
categorias_mas_de_un_registro=categorias_mas_de_un_registro.reset_index()
categorias_mas_de_un_registro=pd.DataFrame(categorias_mas_de_un_registro['index'])
Variables_Correlacion = grupos_variables.merge(categorias_mas_de_un_registro, left_on='Grupo',right_on='index', how='inner')

# seleccioan de la base _0 las variables que presentaron correlación

columnas_seleccionadas = Variables_Correlacion['Variable'].tolist() # Obtener lista de columnas a seleccionar

# Seleccionar columnas en df1 basado en la lista de columnas_seleccionadas
df_seleccionado = df_unido_Normalizada[columnas_seleccionadas]

#Generación de los componentes principales
componentes_p=generar_componentes_principales(df_seleccionado, Variables_Correlacion)

#Borrar columnas corrlacionadas y agregar componentes principales
df_unido_Normalizada_sin_corr=df_unido_Normalizada.drop(columnas_seleccionadas, axis=1)
df_unido_Normalizada_sin_corr=pd.concat([df_unido_Normalizada_sin_corr, componentes_p], axis=1)

In [ ]:
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/df_unido_Normalizada_sin_corr.csv'
df_unido_Normalizada_sin_corr.to_csv(ruta_archivo, index=False)

In [ ]:
df_unido_Normalizada_sin_corr

## 5. Creacion Base 0 Regresión

### 5.1. Integracion de bases de datos

In [ ]:
Base_0=Terridata.merge(Municipios,left_on='Category',right_on='Id_Municipio_Dane')
#Base_0=Base_0.drop(Base_0[Base_0['Year'] == 2021].index)
#Base_0=Base_0.drop(Base_0[Base_0['Year'] == 2022].index)
#Base_0['Year'].value_counts()

###5.2. completar faltantes y eliminar columas 0

In [ ]:
valores_faltantes = Base_0.isna().sum()
valores_faltantes = valores_faltantes[valores_faltantes != 0]
valores_faltantes=pd.DataFrame(valores_faltantes)
valores_faltantes.shape[0]

48

In [ ]:
columnas_a_eliminar = ['departamento_GestComp', 'departamento_MDD','departamento_ResComp','Nombre_Municipio', 'Nombre_Departamento']
Base_0 = Base_0.drop(columnas_a_eliminar, axis=1)
df_filled = df.fillna(0)
Base_0 = Base_0.fillna(0)

In [ ]:
#Elimina columans que sumen 0
sums = Base_0.sum()  # Obtiene la suma de cada columna
zero_sum_columns = sums[sums == 0].index  # Filtra las columnas con suma igual a cero
Base_0=Base_0.drop(zero_sum_columns, axis=1)

pandas.core.indexes.base.Index

In [ ]:
index_series = pd.Series(Base_0.columns)

# Convertir la Serie en un DataFrame
df = index_series.to_frame()

###5.3 Variables calculadas

In [ ]:
Base_0=Base_0.merge(Predial_mes_anterior_3,left_on=['Category','Year'],right_on=['CODIGO_DANE','Año'])
columnas_a_eliminar = ['CODIGO_DANE', 'Año']
Base_0 = Base_0.drop(columnas_a_eliminar, axis=1)
Base_0=Base_0.merge(Predial_mes_anterior_2,left_on=['Category','Year'],right_on=['CODIGO_DANE','Año'])
columnas_a_eliminar = ['CODIGO_DANE', 'Año']
Base_0 = Base_0.drop(columnas_a_eliminar, axis=1)
Base_0=Base_0.merge(Predial_mes_anterior,left_on=['Category','Year'],right_on=['CODIGO_DANE','Año'])
columnas_a_eliminar = ['CODIGO_DANE', 'Año']
Base_0 = Base_0.drop(columnas_a_eliminar, axis=1)
Base_0=Base_0.merge(Predial_Y,left_on=['Category','Year'],right_on=['CODIGO_DANE','Año'],how= "left")
Base_0['Entranamiento'] = Base_0['Crecimiento'].apply(lambda x: 'predicción' if pd.isnull(x) else 'entrenamiento')
columnas_a_eliminar = ['CODIGO_DANE', 'Año']
Base_0 = Base_0.drop(columnas_a_eliminar, axis=1)




In [ ]:
non_numeric_variables = Base_0.select_dtypes(exclude=[np.number]).columns.tolist()
non_numeric_variables

['Entranamiento']

###5.4 Normalizar variables

In [ ]:
# Normalizar variables
lista_cat=['Year', 'Category','Id_Departamento_Dane','Id_Municipio_Dane','Entranamiento','Crecimiento','Crecimiento_mes_anterior','Crecimiento_mes_anterior_2','Crecimiento_mes_anterior_3']
Categorias=Base_0[lista_cat]
Normalizar=Base_0.drop(lista_cat, axis=1)
Base_0_Normalizada=normalize_variables(Normalizar)
Base_0_Normalizada=Categorias.join(Base_0_Normalizada)
#Base_0_Normalizada

###5.5 Simplificacion de variables con correlación alta

In [ ]:
###Analisis de las variables con alta correlación para convertir en componentes principales

#Genera lista con las variables correlacionadas por grupos
lista_cat=['Year', 'Category','Id_Departamento_Dane','Id_Municipio_Dane','Entranamiento']
base_0_corr=Base_0_Normalizada.drop(lista_cat, axis=1)
grupos_variables=obtener_variables_correlacionadas(base_0_corr,0.8)
# Contar las categorías
conteo_categorias = grupos_variables['Grupo'].value_counts()

# Filtrar las categorías con más de un registro
categorias_mas_de_un_registro = conteo_categorias[conteo_categorias > 1]
categorias_mas_de_un_registro=pd.DataFrame(categorias_mas_de_un_registro)
categorias_mas_de_un_registro=categorias_mas_de_un_registro.reset_index()
categorias_mas_de_un_registro=pd.DataFrame(categorias_mas_de_un_registro['index'])
Variables_Correlacion = grupos_variables.merge(categorias_mas_de_un_registro, left_on='Grupo',right_on='index', how='inner')

# seleccioan de la base _0 las variables que presentaron correlación

columnas_seleccionadas = Variables_Correlacion['Variable'].tolist() # Obtener lista de columnas a seleccionar

# Seleccionar columnas en df1 basado en la lista de columnas_seleccionadas
df_seleccionado = Base_0_Normalizada[columnas_seleccionadas]

#Generación de los componentes principales
componentes_p=generar_componentes_principales(df_seleccionado, Variables_Correlacion)

#Borrar columnas corrlacionadas y agregar componentes principales
Base_0_Normalizada_sin_corr=Base_0_Normalizada.drop(columnas_seleccionadas, axis=1)
Base_0_Normalizada_sin_corr=pd.concat([Base_0_Normalizada_sin_corr, componentes_p], axis=1)


In [ ]:
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Base_0_Normalizada.csv'
Base_0_Normalizada_sin_corr.to_csv(ruta_archivo, index=False)